# Menota annotated Völuspá

(AM 242 fol – Codex Wormianus v. 0.9.9) [http://clarino.uib.no/menota/document-element](http://clarino.uib.no/menota/document-element)

To use this notebook, I let you download these texts:

* [Völuspá in Codex Regius](http://clarino.uib.no/menota/texts/GKS-2365-4to-Vsp.xml)
* [Völuspá in Hauksbók](http://clarino.uib.no/menota/texts/AM-544-4to-Vsp.xml)

In [1]:
import os
import codecs
import html
from collections import defaultdict

import xml.etree.ElementTree as ET
from xml.sax.saxutils import escape, unescape

You can open the digitalized manuscripts here...

In [2]:
konungsbok_filename = "GKS-2365-4to-Vsp.xml"
hausbok_filename = "AM-544-4to-Vsp.xml"
menota_directory = "menota"
# #konungsbok = ET.parse(os.path.join(menota_directory, konungsbok_filename))
# # hausbok = ET.parse(os.path.join(menota_directory, hausbok_filename))
# with codecs.open(os.path.join(menota_directory, konungsbok_filename), "r", encoding="utf-8") as f:
#     konungsbok = ET.fromstring(html.unescape(f.read()))
# #    ET.parse
# #with codecs.open(os.path.join(menota_directory, hausbok_filename), "r", encoding="utf-8") as f:
# #     hausbok = ET.fromstring(html.unescape(f.read()))
    
    

... but this does not work because the namespace of tags is not defined.
The **etree** library cannot do that, whereas **lxml** can!

In [3]:
from lxml import etree

With the DTD ([Document Type Definition](https://en.wikipedia.org/wiki/Document_type_definition)) defined inside the document, you can use this text.

In [4]:
parser = etree.XMLParser(load_dtd=True, no_network=False)
tree = etree.parse(os.path.join(menota_directory, konungsbok_filename), parser=parser)

In [5]:
root = tree.getroot()
decoded_root = etree.tostring(root).decode("utf-8")
i = 170
print("\n".join(decoded_root.split("\n")[i:i+10]))

                            <me:dipl>ec</me:dipl>
                        <me:norm>ek</me:norm>
</w>
                        <w me:msa="xAJ rP gF nP cA sI" lemma="allr">
                            <me:facs>allar</me:facs>
                            <me:dipl>allar</me:dipl>
                        <me:norm>allar</me:norm>
</w>
                    </l>
                    <l>


In [6]:
print("\n".join(decoded_root.split("\n")[i:i+30]))

                            <me:dipl>ec</me:dipl>
                        <me:norm>ek</me:norm>
</w>
                        <w me:msa="xAJ rP gF nP cA sI" lemma="allr">
                            <me:facs>allar</me:facs>
                            <me:dipl>allar</me:dipl>
                        <me:norm>allar</me:norm>
</w>
                    </l>
                    <l>
                        <w me:msa="xNC gF nP cA sI" lemma="kind">
                            <me:facs>kin&#42874;ir</me:facs>
                            <me:dipl>kin&#42874;ir</me:dipl>
                        <me:norm>kindir</me:norm>
</w>
                    </l>
                    <l>
                        <w me:msa="xAJ rC gF nP cA sD" lemma="mikill">
                            <me:facs>meiri</me:facs>
                            <me:dipl>meiri</me:dipl>
                        <me:norm>meiri</me:norm>
</w>
                        <w me:msa="xCU" lemma="ok">
                            <me:facs><am>&#826

In [7]:
root.tag

'{http://www.tei-c.org/ns/1.0}TEI'

In [8]:
stringify = etree.XPath("string()")

In [9]:
namespaces = {'n': 'http://www.tei-c.org/ns/1.0',
              'me': 'http://www.menota.org/ns/1.0'}

In [10]:
header, text = root.getchildren()

In [11]:
print(text.find('.//n:div[@type="poem"]', namespaces=namespaces))
poem = text.find('.//n:div[@type="poem"]', namespaces=namespaces)

<Element {http://www.tei-c.org/ns/1.0}div at 0x7fabec240408>


### Extraction of the poem

In [12]:
poem = text.getchildren()[0].getchildren()[3]

In [13]:
stanzas = [child for child in poem.xpath("n:lg", namespaces=namespaces)]

In [14]:
facsimile_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
facsimile_lines = [[line.findall(".//me:facs", namespaces=namespaces) for line in stanza] for stanza in facsimile_stanzas]
facsimile_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in facsimile_lines]

In [15]:
diplomatica_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
diplomatica_lines = [[line.findall(".//me:dipl", namespaces=namespaces) for line in stanza] for stanza in diplomatica_stanzas]
diplomatica_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in diplomatica_lines]

In [16]:
normalized_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
normalized_lines = [[line.findall(".//me:norm", namespaces=namespaces) for line in stanza] for stanza in normalized_stanzas]
normalized_text = [[[stringify(word) for word in line] for line in stanza ] for stanza in normalized_lines]

In [17]:
lemmata_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
lemmata_lines = [[line.findall(".//n:w", namespaces=namespaces) for line in stanza] for stanza in lemmata_stanzas]
lemmata_text = [[[word.get("lemma") for word in line] for line in stanza ] for stanza in lemmata_lines]

In [18]:
pos_stanzas = [stanza.findall(".//n:l", namespaces=namespaces) for stanza in stanzas]
pos_lines = [[line.findall(".//n:w", namespaces=namespaces) for line in stanza] for stanza in pos_stanzas]
pos_text = [[[word.get('{http://www.menota.org/ns/1.0}msa') for word in line] for line in stanza ] for stanza in pos_lines]

#### First stanza

In [19]:
facsimile_text[0]

[['Hlıoðſ', 'bið', 'ec', 'allar'],
 ['kinꝺir'],
 ['meiri', '⁊', 'miɴi'],
 ['maugo', 'heimꝺalar'],
 ['uilðo', 'at', 'ec', 'ualꝼꜹþ̅'],
 ['uel', 'ꝼyr', 'telia'],
 ['ꝼoꝛn', 'ſpioll', 'ꝼíra'],
 ['þꜹ', 'e͛', 'ꝼremſt', 'u̅', 'man', '.']]

In [20]:
diplomatica_text[0]

[['Hlıoðſ', 'bið', 'ec', 'allar'],
 ['kinꝺir'],
 ['meiri', 'oc', 'miɴi'],
 ['maugo', 'heimꝺalar'],
 ['uilðo', 'at', 'ec', 'ualꝼꜹþr'],
 ['uel', 'ꝼyr', 'telia'],
 ['ꝼoꝛn', 'ſpioll', 'ꝼíra'],
 ['þꜹ', 'er', 'ꝼremſt', 'um', 'man', '.']]

In [21]:
normalized_text[0]

[['Hljóðs', 'bið', 'ek', 'allar'],
 ['kindir'],
 ['meiri', 'ok', 'minni'],
 ['mögu', 'Heimdalar'],
 ['vildu', 'að', 'ek', 'Valföðr'],
 ['vel', 'fyr', 'telja'],
 ['forn', 'spjöll', 'fira'],
 ['þau', 'er', 'fremst', 'um', 'man', '.']]

In [22]:
lemmata_text[0]

[['hlióð', 'biðia', 'ek', 'allr'],
 ['kind'],
 ['mikill', 'ok', 'lítill'],
 ['mǫgr', 'Heimdallr'],
 ['vilia', 'at', 'ek', 'Valfǫðr'],
 ['vel', 'fyr', 'telia'],
 ['forn', 'spiall', 'firar'],
 ['sá', 'er', 'framr', 'um', 'muna']]

In [23]:
pos_text[0]

[['xNC gN nS cG sI',
  'xVB fF tPS mIN p1 nS vA iST',
  'xPD cN',
  'xAJ rP gF nP cA sI'],
 ['xNC gF nP cA sI'],
 ['xAJ rC gF nP cA sD', 'xCU', 'xAJ rC gF nP cA sD'],
 ['xNC gM nP cA sI', 'xNP gM cG sI'],
 ['xVB fF tPS mIN p2 nS vA iWK', 'xCU', 'xPD cN', 'xNC gM nS cN'],
 ['xAV rP', 'xAV rP', 'xVB fF tPS mSU p1 nS vA iWK'],
 ['xAJ rP gN nP cA sI', 'xNC gN nP cA sI', 'xNC gM nP cG sI'],
 ['xPD gN nP cA',
  'xCU',
  'xAJ rS gN nP cA sI',
  'xAV rP',
  'xVB fF tPS mIN p1 nS vA iPP']]

### Parsing the POS tags

These annotations are ununderstandable without the docs that you can find [here](https://menota.org/HB3_ch11.xml#sec11.3). The idea is to parse these annotations the same way I did in https://github.com/cltk/old_norse_texts_heimskringla/blob/master/pos.py

In [24]:
class WordClass:
    identifier = "x"
    noun = "N"
    common_noun = "NC"
    proper_noun = "NP"
    adjective = "AJ"
    pronoun = "P"
    personal_pronoun = "PE"
    interrogative_pronoun = "PQ"
    indefinite_pronouns = "PI"
    possessive = "DP"
    demonstrative = "DD"
    quantifier = "DQ"
    pronoun = "PD"
    number = "N"
    cardinal = "NA"
    ordinal = "NO"
    numeral_undetermined = "NU"
    article = "AT"
    verb = "VB"
    adverb = "AV"
    preposition = "VP"
    adposition = "AP"
    conjunction = "CC"
    subjunction = "CS"
    conjunction_subjunction = "CU"
    interjection = "IT"
    infinitive_marker = "IM"
    relative_particle = "RP"
    expletive_particle = "EX"
    unassigned = "UA"

In [35]:
class Gender:
    identifier = "g"
    masculine = "M"
    feminine = "F"
    neuter = "N"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[masculine] = "masculine"
    verbose[feminine] = "feminine"
    verbose[neuter] = "neuter"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Gender.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Gender.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.masculine = tag == Gender.masculine
        value.feminine = tag == Gender.feminine
        value.neuter = tag == Gender.neuter
        return value
    
class Number:
    identifier = "n"
    singular = "S"
    dual = "D"
    plural = "P"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[singular] = "singular"
    verbose[plural] = "plural"
    verbose[dual] = "dual"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Number.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Number.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.singular = tag == Number.singular
        value.plural = tag == Number.plural
        value.dual = tag == Number.dual
        return value
        
    
class Case:
    identifier = "c"
    nominative = "N"
    genitive = "G"
    dative = "D"
    accusative = "A"
    oblique = "O"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[nominative] = "nominative"
    verbose[accusative] = "feminine"
    verbose[genitive] = "genitive"
    verbose[dative] = "dative"
    verbose[oblique] = "oblique"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Case.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Case.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.nominative = tag == Case.nominative
        value.accusative = tag == Case.accusative
        value.dative = tag == Case.dative
        value.genitive = tag == Case.genitive
        return value
    
class Species:
    identifier = "s"
    indefinite = "I"
    definite = "D"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[indefinite] = "indefinite"
    verbose[definite] = "definite"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Species.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Species.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.definite = tag == Species.definite
        value.indefinite = tag == Species.indefinite
        return value
    
class Grade:
    identifier = "r"
    positive = "P"
    comparative = "C"
    superlative = "S"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[positive] = "positive"
    verbose[comparative] = "comparative"
    verbose[superlative] = "superlative"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Grade.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Grade.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.positive = tag == Grade.positive
        value.comparative = tag == Grade.comparative
        value.superlative = tag == Grade.superlative
        return value
    
class Person:
    identifier = "p"
    first = "1"
    second = "2"
    third = "3"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[first] = "first"
    verbose[second] = "second"
    verbose[third] = "third"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Person.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Person.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.first = tag == Person.first
        value.second = tag == Person.second
        value.third = tag == Person.third
        return value

class Tense:
    identifier = "t"
    present = "PS"
    preterite = "PT"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[present] = "present"
    verbose[preterite] = "preterite"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Tense.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Tense.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.present = tag == Tense.present
        value.preterite = tag == Tense.preterite
        return value
    
class Mood:
    identifier = "m"
    indicative = "IN"
    subjunctive = "SU"
    imperative = "IP"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[indicative] = "indicative"
    verbose[subjunctive] = "subjunctive"
    verbose[imperative] = "imperative"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Mood.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Mood.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.indicative = tag == Mood.indicative
        value.subjunctive = tag == Mood.subjunctive
        value.imperative = tag == Mood.imperative
        return value
    
class Voice:
    identifier = "v"
    active = "A"
    reflexive = "R"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[active] = "active"
    verbose[reflexive] = "reflexive"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Voice.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Voice.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.active = tag == Voice.active
        value.reflexive = tag == Voice.reflexive
        return value
    
class Finitness:
    identifier = "f"
    finite = "F"
    infinite = "I"
    participle = "P"
    unspecified = "U"
    
    verbose = defaultdict(str)
    verbose[finite] = "finite"
    verbose[infinite] = "infinite"
    verbose[participle] = "participle"
    verbose[unspecified] = "unspecified"

    @staticmethod
    def parse(tag, value):
        return value + " " + Finitness.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Finitness.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.finite = tag == Finitness.finite
        value.infinite = tag == Finitness.infinite
        value.participle = tag == Finitness.participle
        return value
    
class InflectionalClass:
    identifier = "i"
    strong = "ST"
    weak = "WK"
    reduplicating = "RD"
    preterito_present = "PP"
    
    verbose = defaultdict(str)
    verbose[strong] = "strong"
    verbose[weak] = "weak"
    verbose[reduplicating] = "reduplicating"
    verbose[preterito_present] = "preterito-present"

    @staticmethod
    def parse(tag, value):
        return value + " " + InflectionalClass.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in InflectionalClass.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.strong = tag == InflectionalClass.strong
        value.weak = tag == InflectionalClass.weak
        value.reduplicating = tag == InflectionalClass.reduplicating
        value.preterito_present = tag == InflectionalClass.preterito_present
        return value
    
class Enclitic:
    identifier = "e"
    pronoun = "P"
    negative_particle = "N"
    
    verbose = defaultdict(str)
    verbose[pronoun] = "pronoun"
    verbose[negative_particle] = "negative particle"

    @staticmethod
    def parse(tag, value):
        return value + " " + Enclitic.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Enclitic.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.pronoun = tag == Enclitic.pronoun
        value.negative_particle = tag == Enclitic.negative_particle
        return value
    
class Government:
    identifier = "y"
    genitive = "G"
    dative = "D"
    accusative = "A"
    unspecified = "U"
    indicative = "UN"
    subjunctive = "SU"
    
    verbose = defaultdict(str)
    verbose[genitive] = "genitive"
    verbose[dative] = "dative"
    verbose[accusative] = "accusative"
    verbose[indicative] = "indicative"
    verbose[subjunctive] = "subjunctive"
    verbose[unspecified] = "unspecified gender"

    @staticmethod
    def parse(tag, value):
        return value + " " + Government.verbose[tag]

    @staticmethod
    def can_apply(tag):
        return tag in Government.verbose
    
    @staticmethod
    def binarize(tag, value):
        value.genitive = tag == Government.genitive
        value.dative = tag == Government.dative
        value.accusative = tag == Government.accusative
        value.indicative = tag == Government.indicative
        value.subjunctive = tag == Government.subjunctive
        return value

    
DELIMITER = "|"

In [56]:
class POSFeatures:
    
    def __init__(self):
        self.masculine = False
        self.feminine = False
        self.neuter = False
        self.singular = False
        self.plural = False
        self.dual = False
        self.nominative = False
        self.accusative = False
        self.dative = False
        self.definite = False
        self.indefinite = False
        self.positive = False
        self.comparative = False
        self.superlative = False
        self.first = False
        self.second = False
        self.third = False
        self.indicative = False
        self.subjunctive = False
        self.imperative = False
        self.present = False
        self.preterite = False
        self.active = False
        self.reflexive = False
        self.infinitive = False
        self.participle = False
        self.strong = False
        self.weak = False
        self.reduplicating = False
        self.preterito_present = False
        self.noun = False
        self.adjective = False
        self.article = False
        self.numeral = False
        self.verb = False
        self.adverb = False
        self.conjunction = False
        self.foreign = False
        self.punctuation = False
        self.unanalysed = False
        
    def __eq__(self, other):
        for i in range(len(self.vectorize())):
            if self.vectorize()[i] != other.vectorize()[i]:
                return False
        return True
        
    def vectorize(self):
        return [
            self.masculine,
            self.feminine,
            self.neuter,
            self.singular,
            self.plural,
            self.dual,
            self.nominative,
            self.accusative,
            self.dative,
            self.definite,
            self.indefinite,
            self.positive,
            self.comparative,
            self.superlative,
            self.first,
            self.second,
            self.third,
            self.indicative,
            self.subjunctive,
            self.imperative,
            self.present,
            self.preterite,
            self.active,
            self.reflexive,
            self.infinitive,
            self.participle,
            self.strong,
            self.weak,
            self.reduplicating,
            self.preterito_present,
            self.noun,
            self.adjective,
            self.article,
            self.numeral,
            self.verb,
            self.adverb,
            self.foreign,
            self.punctuation,
            self.unanalysed
        ]

In [49]:
from collections import defaultdict

class MainPOS:
    noun = "n"
    adjective = "l"
    pronoun = "f"
    article = "g"
    numeral = "t"
    verb = "s"
    adverb = "a"
    conjunction = "c"
    foreign = "e"
    unanalysed = "x"
    punctuation = "p"

    verbose = defaultdict(str)
    verbose[WordClass.common_noun] = "common noun"
    verbose[WordClass.proper_noun] = "proper noun"
    verbose[WordClass.adjective] = "adjective"
    verbose[WordClass.pronoun] = "pronoun"
    verbose[WordClass.article] = "article"
    verbose[WordClass.ordinal] = "ordinal"
    verbose[WordClass.cardinal] = "cardinal"
    verbose[WordClass.numeral_undetermined] = "numeral undetermined"
    verbose[WordClass.verb] = "verb"
    verbose[WordClass.adverb] = "adverb"
    verbose[WordClass.conjunction] = "conjunction"
    verbose[WordClass.subjunction] = "subjunction"
    verbose[WordClass.conjunction_subjunction] = "conjunction or subjunction"
    verbose[WordClass.unassigned] = "unanalysed"
    verbose[WordClass.unassigned] = "unanalysed"
#     verbose[WordClass.punctuation] = "punctuation"
#     verbose[WordClass.foreign] = "foreign"

#     icepac = defaultdict(str)
#     icepac[WordClass.comm]

    # A universal part-of-speech tagset by Petrov S., Das D., McDonald R.

    universal = defaultdict(str)
    universal[noun] = "NOUN"
    universal[adjective] = "ADJ"
    universal[pronoun] = "PRON"
    universal[article] = "DET"
    universal[numeral] = "NUM"
    universal[verb] = "VERB"
    universal[adverb] = "ADV"  # in this category, belong ADV, ADP and PRT
    universal[conjunction] = "CONJ"
    universal[unanalysed] = "X"
    universal[foreign] = "X"
    universal[punctuation] = "PUNC"

    @staticmethod
    def apply(tag: list, l_pos: list, value: str):
#         print(tag)
        i = 0
        for pos in l_pos:
#             print(pos)
            new_value = ""
            if isinstance(pos, list):
                for j in pos:
                    if j.can_apply(tag[i][1:]):
                        new_value = j.parse(tag[i], value)
            elif i < len(tag):
#                 print(tag[i])
                new_value = pos.parse(tag[i][1:], value)
#                 print(repr(value))
            if new_value != "" and value+" " != new_value:
                value = new_value
                i += 1
        return value
    
    @staticmethod
    def binarize(tag: list, l_pos: list, value: POSFeatures):
#         print(tag)
        i = 0
        for pos in l_pos:
            print(pos)
            if isinstance(pos, list):
                for j in pos:
                    if j.can_apply(tag[i][1:]):
                        new_value = j.binarize(tag[i], value)
            elif i < len(tag):
#                 print(tag[i])
                new_value = pos.binarize(tag[i][1:], value)
#                 print(repr(value))
            value = new_value
            i += 1
        return value

    @staticmethod
    def parse(full_tag):
        """
        >>> MainPOS.parse('xPE p1 nD cN')
        
        :param tag:
        :return:
        """
        features = POSFeatures()
        tags = full_tag.split(" ")
        
        first_tag = tags[0][0]
        if WordClass.identifier == first_tag:
            word_class = tags[0][1:]
#             print(word_class)
            value = ""
            tag = tags[1:]
            if word_class == WordClass.noun:
                features.noun = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.proper_noun:
                features.proper_noun = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.common_noun:
                features.common_noun = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)

            elif word_class == WordClass.adjective:
                features.adjective = True
                value = MainPOS.verbose[word_class]
                parsers = [Grade, Gender, Number, Case, Species, Grade]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)

            elif word_class == WordClass.personal_pronoun:
                features.personal_pronoun = True
                value = MainPOS.verbose[word_class]
                parsers = [Person, Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.interrogative_pronoun:
                features.interrogative_pronoun = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.indefinite_pronouns:
                features.indefinite_pronouns = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.possessive:
                features.possessive = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.demonstrative:
                features.demonstrative = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.quantifier:
                features.quantifier = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.article:
                features.article = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
            
            elif word_class == WordClass.pronoun:
                features.pronoun = True
                value = MainPOS.verbose[word_class]
                parsers = [Person, Gender, Number, Case]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)

            elif word_class == WordClass.ordinal:
                features.ordinal = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.cardinal:
                features.cardinal = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.numeral_undetermined:
                features.numeral = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.article:
                features.article = True
                value = MainPOS.verbose[word_class]
                parsers = [Gender, Number, Case, Species]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)

            elif word_class == WordClass.verb:
                features.verb = True
                value = MainPOS.verbose[word_class]
                verb_class = tags[1][1]
                if verb_class == "F":
                    parsers = [Finitness, Tense, Mood, Person, Number, Voice, InflectionalClass]
                    value = MainPOS.apply(tag, parsers, value)
                    features = MainPOS.binarize(tag, parsers, features)
                elif verb_class == "P":
                    parsers = [Finitness, Tense, Voice, Gender, Number, Case, Species, InflectionalClass]
                    value = MainPOS.apply(tag, parsers, value)
                    features = MainPOS.binarize(tag, parsers, features)
                elif verb_class == "I":
                    parsers = [Finitness, Tense, Voice, InflectionalClass]
                    value = MainPOS.apply(tag, parsers, value)
                    features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.adverb:
                features.adverb = True
                value = MainPOS.verbose[word_class]
                parsers = [Grade]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.preposition:
                features.preposition = True
                value = MainPOS.verbose[word_class]
                parsers = [Government]
                value = MainPOS.apply(tag, parsers, value)
                features = MainPOS.binarize(tag, parsers, features)
                
            elif word_class == WordClass.adposition:
                value = MainPOS.verbose[word_class]
                
                
            elif word_class == WordClass.conjunction:
                value = MainPOS.verbose[word_class]
                
            elif word_class == WordClass.conjunction_subjunction:
                value = MainPOS.verbose[word_class]
            
            elif word_class == WordClass.subjunction:
                value = MainPOS.verbose[word_class]
                
            return value, features


def parse(tag):
#     print(tag)
    if len(tag) > 0:
        value, features = MainPOS.parse(tag)
    else:
        value = ""
    return value, features

In [57]:
a, b = parse("xVB fP tPT vA gM nS cN sI")
b.vectorize()

<class '__main__.Finitness'>
<class '__main__.Tense'>
<class '__main__.Voice'>
<class '__main__.Gender'>
<class '__main__.Number'>
<class '__main__.Case'>
<class '__main__.Species'>
<class '__main__.InflectionalClass'>


[True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False]

In [58]:
parse("xNC gM nP cD sI") # hestum

<class '__main__.Gender'>
<class '__main__.Number'>
<class '__main__.Case'>
<class '__main__.Species'>


('common noun masculine plural dative indefinite',
 <__main__.POSFeatures at 0x7fabe4278d30>)

In [59]:
for i, stanza in enumerate([pos_text[0]]):
    for j, line in enumerate(stanza):
        for k, pos in enumerate(line):
            print(normalized_text[0][j][k]+" ["+lemmata_text[0][j][k]+"]"+" -> "+parse(pos)+" : "+pos)

<class '__main__.Gender'>
<class '__main__.Number'>
<class '__main__.Case'>
<class '__main__.Species'>


TypeError: must be str, not tuple

This notebook will become a repository of CLTK.

By Clément Besnier, (visit my [website](https://www.clementbesnier.fr))